# GCN for DAVIS 2016

In this notebook, a custom [PyTorch Geometric](https://rusty1s.github.io/pytorch_geometric/build/html/index.html) [InMemoryDataset](https://rusty1s.github.io/pytorch_geometric/build/html/_modules/torch_geometric/data/in_memory_dataset.html#InMemoryDataset) for the DAVIS 2016 dataset is created. The implementation is based on this [tutorial](https://rusty1s.github.io/pytorch_geometric/build/html/notes/create_dataset.html). The dataset is then used to train a simple GCN network as a first evaluation based on this [tutorial](https://rusty1s.github.io/pytorch_geometric/build/html/notes/introduction.html#learning-methods-on-graphs).

The dataset consists of single PyTorch Geometric [Data](https://rusty1s.github.io/pytorch_geometric/build/html/_modules/torch_geometric/data/data.html#Data) objects which model a single graph with various attributes. For this dataset, a graph for each contour is created. Hereby, each node of the graph represents one contour point. The feature of each node is the OSVOS feature vector from the next frame at this point. Each node is connected to its K nearest neighbours. The feature of each edge is the distance between the nodes it connects. The targets of each node is the translation it undergoes from the current to the next frame.

## Imports

In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

from pg_datasets.davis_2016 import DAVIS2016

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Paths & Constants

In [2]:
PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH = 'pg_datasets/DAVIS_2016'
CONTOURS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Contours/480p'
IMAGES_FOLDERS_PATH = 'DAVIS_2016/DAVIS/JPEGImages/480p'
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'

LAYER = 9
K = 32

SKIP_SEQUENCES = ['bmx-trees', 'bus', 'cows', 'dog-agility', 'horsejump-high', 
                  'horsejump-low', 'kite-walk', 'lucia', 'libby', 'motorbike',
                  'paragliding', 'rhino', 'scooter-gray', 'swing']

TRAIN_SEQUENCES = ['bear', 'bmx-bumps', 'boat', 'breakdance-flare', 'bus', 
                   'car-turn', 'dance-jump', 'dog-agility', 'drift-turn', 
                   'elephant', 'flamingo', 'hike', 'hockey', 'horsejump-low', 
                   'kite-walk', 'lucia', 'mallard-fly', 'mallard-water', 
                   'motocross-bumps', 'motorbike', 'paragliding', 'rhino', 
                   'rollerblade', 'scooter-gray', 'soccerball', 'stroller',
                   'surf', 'swing', 'tennis', 'train']

VAL_SEQUENCES = ['blackswan', 'bmx-trees', 'breakdance', 'camel', 'car-roundabout',
                 'car-shadow', 'cows', 'dance-twirl', 'dog', 'drift-chicane', 
                 'drift-straight', 'goat', 'horsejump-high', 'kite-surf', 'libby', 
                 'motocross-jump', 'paragliding-launch', 'parkour', 'scooter-black', 
                 'soapbox']

## Dataset

In [3]:
dataset = DAVIS2016(PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH, 
                    CONTOURS_FOLDERS_PATH, IMAGES_FOLDERS_PATH, TRANSLATIONS_FOLDERS_PATH, 
                    LAYER, K, 
                    SKIP_SEQUENCES, TRAIN_SEQUENCES, VAL_SEQUENCES)

Processing...
#0: bear
Start online training...
Finished online training...
Create new OSVOS model...
Constructing OSVOS architecture..
Initializing weights..
#0: 00000.npy
#1: 00001.npy
#2: 00002.npy
#3: 00003.npy
#4: 00004.npy
#5: 00005.npy
#6: 00006.npy
#7: 00007.npy
#8: 00008.npy
#9: 00009.npy
#10: 00010.npy
#11: 00011.npy
#12: 00012.npy
#13: 00013.npy
#14: 00014.npy
#15: 00015.npy
#16: 00016.npy
#17: 00017.npy
#18: 00018.npy
#19: 00019.npy
#20: 00020.npy
#21: 00021.npy
#22: 00022.npy
#23: 00023.npy
#24: 00024.npy
#25: 00025.npy
#26: 00026.npy
#27: 00027.npy
#28: 00028.npy
#29: 00029.npy
#30: 00030.npy
#31: 00031.npy
#32: 00032.npy
#33: 00033.npy
#34: 00034.npy
#35: 00035.npy
#36: 00036.npy
#37: 00037.npy
#38: 00038.npy
#39: 00039.npy
#40: 00040.npy
#41: 00041.npy
#42: 00042.npy
#43: 00043.npy
#44: 00044.npy
#45: 00045.npy
#46: 00046.npy
#47: 00047.npy
#48: 00048.npy
#49: 00049.npy
#50: 00050.npy
#51: 00051.npy
#52: 00052.npy
#53: 00053.npy
#54: 00054.npy
#55: 00055.npy
#56: 00056.

IndexError: index 240 is out of bounds for dimension 2 with size 240

In [ ]:
data = dataset[0]
for key, item in data:
    print(key, type(item))

print(data)

## Simple GCN

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [ ]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))